# <font color='#28B463'>Bootstrapping 

<br>

## <font color='#28B463'>Introduction

<br>
The bootstrap method is a re-sampling technique for estimating a sampling distribution. <b>The idea behind the bootstrap is to generate new data from a population by repeated sampling from the original dataset with replacement</b>; in contrast, the repeated hold-out method can be understood as sampling without replacement.

<br>
In this notebook, we will see how the bootstrap method can help us to determine the statistical properties of an estimator (classification accuracy,in this case) when the underlying distribution of the data is unknown and additional samples are not available.

## <font color='#28B463'>In theory

<br>
Given a dataset of size $N$ :

<br>
<ul style="list-style-type:square">
    <li>
        for $b$ bootstrap rounds we draw one single instance from the dataset and assign it to our $j^{th}$ bootstrap sample,
        and we repeat this step until our bootstrap sample has the same size of the original dataset; <b>the data points are
        always drawn from the original dataset with replacement, so that certain data points may appear more than once in our
        bootstrap sample and some not at all</b>
    </li>
    <br>
    <li>
        <b>we fit a model to each of the $b$ bootstrap samples</b> and evaluate on that same training set (resubstitution
        accuracy)
    </li>
    <br>
    <li>
        the model accuracy is computed as the average over the $b$ accuracy estimates
    </li>
</ul>

<br>
As we already discussed in a previous notebook, the <b>resubstitution accuracy usually leads to an extremely optimistic bias</b>.


## <font color='#28B463'>In practice

<br>
In order to exploit this method for the evaluation of predictive models, such as hypotheses for classification and regression, we may prefer a slightly different approach to bootstrapping using the so-called <b>Leave-One-Out Bootstrap (LOOB)</b> technique. 

<br>
Instead of evaluating the model on the training data, we use <b>out-of-bag samples as test sets for evaluation</b>. These out-of-bag samples are the unique sets of instances that are not used for model fitting.


<img src="images/bootstrap-summary.png" width="55%" height="55%">

<br>
A number of bootstrap samples between 50 and 200 is usually considered as being sufficient for reliable estimates.

<br>
$
    \quad
    \begin{align}
        ACC_{boot} &= \frac{1}{b} \sum_{i = 1}^{b} ACC_i
        \newline \newline
        SE_{boot} &= \sqrt{ \frac{1}{b-1} \sum_{i=1}^{b} (ACC_i - ACC_{boot})^2 }
    \end{align}
$

## <font color='#28B463'>The .632 Estimate

<br>
In 1983, Bradley Efron introduced the <b>.632 Estimate, a further improvement to address the pessimistic bias of the bootstrap cross-validation approach</b> described above. The pessimistic bias in the "classic" bootstrap method can be attributed to the fact that the bootstrap samples only contain approximately 63.2% of the unique samples from the original dataset.


<br>
We can compute the probability that a given sample from a dataset of size $N$ is not drawn as a bootstrap sample as :

<br>
$
    \quad
    P (\text{not chosen}) =  \bigg(1 - \frac{1}{N}\bigg)^N
    \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \quad 
    \text{which is asymptotically equivalent to } \frac{1}{e} \approx 0.3681 \text{ as } N \rightarrow \infty
    \\
    \quad
    P (\text{chosen}) = 1 - \bigg(1 - \frac{1}{N}\bigg)^N \approx 0.632 
$

<img src="images/bootstrap-632-prob.png" width="45%" height="45%">

<br>
For reasonably large datasets, we can expect approximately $0.632 \cdot N$ uniques samples to be selected for any of the $b$ bootstrap training sets; accordingly, we can expect $0.368 \cdot N$ out-of-bag samples to be available for testing in each iteration.

<br>
In order to address the bias that is due to sampling with replacement, Bradley Efron proposed the .632 Estimate, which is computed via the following equation :

<br>
$
    \quad
    \text{ACC}_{boot} = \frac{1}{b} \sum_{i=1}^b \big(0.632 \cdot \text{ACC}_{h, j} + 0.368 \cdot \text{ACC}_{r, j}\big),
$

<br>
where $\text{ACC}_{r, j}$ is the resubstitution accuracy of the $j^{th}$ bootstrap sample and $\text{ACC}_{h, j}$ is the accuracy on the out-of-bag sample.

<br>
<b>Although the <i>.632 Boostrap</i> attempts to address the pessimistic bias of the estimate, an optimistic bias may occur with models that tend to overfit; in 1997 Efron and Tibshirani proposed the <i>.632+ Bootstrap Method</i> </b>: instead of using a "fixed weight" ($\omega = .632$) in the formula above, the latter can be rephrased as :

<br>
$
    \quad
    \text{ACC}_{boot} 
    = \frac{1}{b} \sum_{i=1}^b \big( \omega \cdot \text{ACC}_{h, i} + (1 - \omega) \cdot \text{ACC}_{r, i} \big),
$

<br>
where the weight is obtained from the relative overfitting rate $R$ : 

<br>
$
    \quad
    \begin{align}
        \omega &= \frac{0.632}{1 - 0.368 \cdot R}
        \newline \newline
        R &= (-1) \cdot \frac{ (\text{ACC}_{h, i} - \text{ACC}_{r, i}) } {\gamma - (1 -\text{ACC}_{h, i})}
    \end{align}
$

<br>
In order to compute the relative overfitting rate $R$ we first have to determine the no-information rate $\gamma$; we can compute $\gamma$ by fitting a model to a dataset that contains all possible combinations between samples $\boldsymbol{X_I}$ and target class labels $\boldsymbol{Y_i}$ (and pretend that observations and class labels are independent), or we can estimate it using the second formula :

<br>
$
    \quad
    \begin{align}
        \gamma &= \frac{1}{n^2} \sum_{i = 1}^{n} \sum_{i = 1}^{n} L(\boldsymbol{Y_i}, f(\boldsymbol{X_i}))
        \newline
        \gamma &= \sum_{k = 1}^K p_k \ (1 - q_k)
    \end{align}
$

<br>
where $p_k$ is the proportion of class $k$ samples observed in the dataset, and $q_k$ is the proportion of class $k$ samples that the classifier predicts in the dataset.

## <font color='#28B463'>References

<br>
<ul style="list-style-type:square">
    <li>
        Sebastian Raschka - Model evaluation, model selection, and algorithm selection in machine learning - Part III <br>
        https://sebastianraschka.com/blog/2016/model-evaluation-selection-part3.html
    </li>
</ul>